In [33]:
import pandas as pd
import numpy as np

In [8]:
raw_data = pd.read_json("../get_rankings/data/rankings_list.json")

##Number of times position shows up

In [10]:
raw_data.position.value_counts()

RB    113
WR    103
TE     35
QB     32
K      19
D      18
dtype: int64

##Metrics on score by position
Looks like kickers score a lot, but almost no variance. Can see why RB's get taken first. Highest varience, so don't want to get stuck with a bad one

In [15]:
raw_data.groupby("position").points.mean().order(ascending=False)

position
QB    245.050000
K     132.947368
D     114.333333
WR     98.866990
RB     82.410619
TE     79.620000
Name: points, dtype: float64

In [68]:
raw_data.groupby("position").points.std().order(ascending=False)

position
RB    68.318816
QB    50.485654
WR    46.053046
TE    29.101211
D     22.857616
K      5.641844
Name: points, dtype: float64

##ADP
Lets use these counts for my baseline

In [17]:
adp = pd.read_csv("../get_rankings/data/adp.csv")

In [18]:
adp.head()

,ADP,Overall,Name,Position,Team,Times Drafted,Bye
0,1.02,2.1,Adrian Peterson,RB,MIN,254,5
1,1.03,2.5,LeVeon Bell,RB,PIT,127,11
2,1.04,3.7,Eddie Lacy,RB,GB,295,7
3,1.04,3.9,Jamaal Charles,RB,KC,160,9
4,1.05,5.1,Marshawn Lynch,RB,SEA,184,9


In [19]:
adp.Position.value_counts()

WR     61
RB     57
QB     20
TE     17
DEF    11
PK      6
dtype: int64

##Calculate Added Value

In [42]:
positions = ['WR', 'RB', 'QB', 'TE', 'D', 'K']
baseline = [61, 57, 20, 17, 12, 12]

In [51]:
def get_adj_points(pos, max_n):
    position_data = raw_data[raw_data.position == pos].sort("points", ascending=False)
    position_data = position_data.set_index(np.arange(position_data.shape[0]))
    position_data = position_data[position_data.index < max_n]
    position_data['value_adj_points'] = position_data.points - position_data.iloc[max_n - 1].points
    return position_data

In [53]:
all_adj = []
for i in range(len(positions)):
    all_adj.append(get_adj_points(positions[i], baseline[i]))

In [58]:
all_adj_df = pd.concat(all_adj)
all_adj_df = all_adj_df.sort("value_adj_points", ascending=False)
all_adj_df.head()

,name,points,position,team,value_adj_points
0,Eddie Lacy,232.7,RB,GB,165.5
1,Adrian Peterson,222.6,RB,Min,155.4
2,Jamaal Charles,218.7,RB,KC,151.5
3,C.J. Anderson,218.6,RB,Den,151.4
4,Marshawn Lynch,216.6,RB,Sea,149.4


##Merge on ADP

In [60]:
adp_sub = adp[['ADP', 'Name', 'Bye']]

In [62]:
final_df = pd.merge(all_adj_df, adp_sub, left_on='name', right_on='Name', how='left')

In [66]:
final_df.to_csv("data/my_rankings.csv")

In [72]:
final_df.groupby('position').points.std().order(ascending=False)

position
RB    49.020958
WR    31.320775
QB    24.856097
TE    22.505387
D     22.407385
K      3.146667
Name: points, dtype: float64

In [73]:
final_df.groupby('position').points.mean().order(ascending=False)

position
QB    275.335000
RB    138.852632
K     136.583333
WR    129.306557
D     123.500000
TE    100.364706
Name: points, dtype: float64